In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [18]:
%run -i ../../core.py

In [19]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [20]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.div(1.0,torch.mean(torch.div(1.0,dummy_data_tensor)))

In [21]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<=torch.abs(0.1*X.size()[1]), self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [22]:
# prover calculates settings, send to verifier
theory_output = torch.div(1.0,torch.mean(torch.div(1.0,data_tensor)))
print("Theory_output: ",theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<=torch.abs(0.1*X.size()[1]), self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [6], "resources", settings_path)

Theory_output:  tensor(45.2144)
==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-184392,11264],"logrows":18,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":911,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Recip":{"scale":4096.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [23]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 24.533974170684814 seconds
Theory output:  tensor(45.2144)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]], 'proof': '2269876ecef415ad55830ccb4c25b99dde635dee8c7da12684a29ea40569dc351a4229ae31099261db68d955bb23546be3aa416b71c9ab24fa2e18493dbc67002b1e4bc3933809133829991237de759d3c9158d64fd5f713cdeab7da30f5a76e15ae2be48093d5528ca3213c13b1fb5bcea0cff65dcb24b90a51098932b20d8001d652453e6d9f108422dbc9d40d680aa3aefc787c0faaa16b76f1c54e7b266e12006255adc376c1bd522c4f98a61201c7e5866e8f284fdc47bc2f7fc4e8e4640d3232763ef3864e8a9c9e708987ac242c4064ccc39eae367dc07a1ba439b5c41497570e2c0ff513b7ca7859140d932103901c2412d901be7bec14280dd7dc13184aefe66b89eebad1eedd1666189ab429bbfdd384f4415d5caf82f77b7528f21809713b388469a459b617c220ad8377f2a3fff9739ac99a0a180001770cdcd829799e8e038990ea4fc6c0d04429af0a20e3a3e19b5254872859f2b74ce9

In [24]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]]
proof boolean:  1.0
proof result 1 : 45.21875
verified
